<h1 style='font-size:40px'> Apache Spark's Structured APIs</h1>

<h2 style='font-size:30px'> Key Merits and Beneftis</h2>

In [7]:
# Criando o nosso primeiro DataFrame dentro do Spark.
# Vamos aproveitar a situação e fazer algumas operações.
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg
spark = SparkSession.builder.appName('Chapter3').getOrCreate()

# O método createDataFrame demanda uma lista de tuplas para que cada linha seja construída.
# Como segundo argumento, uma lista com o nome das colunas pode ser passada.
data_df = spark.createDataFrame([('Brooke', 20), ('Denny', 31), ('Jules', 30),
                                ('TD', 35), ('Brooke', 25)],['name', 'age'])

data_df.show()

+------+---+
|  name|age|
+------+---+
|Brooke| 20|
| Denny| 31|
| Jules| 30|
|    TD| 35|
|Brooke| 25|
+------+---+



In [9]:
# Agrupando os dados conforme o nome e extraindo a idade média.

# A sintaxe é um pouco diferente da do pandas, o método agg com um outro método de agregação como argumento deve acompahar 'groupBy'.
data_df.groupBy('name').agg(avg('age')).show()

+------+--------+
|  name|avg(age)|
+------+--------+
|Brooke|    22.5|
| Denny|    31.0|
| Jules|    30.0|
|    TD|    35.0|
+------+--------+



<h2 style='font-size:30px'> The DataFrame API</h2>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Os DataFrames do PySpark são como os do pandas. Têm estrutura de tabelas e permitem a nós fazermos operações sobre os seus dados.
        </li>
    </ul>
</div>

<h3 style='font-size:30px;font-style:italic'> Spark's Basic Data Types</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            O Spark possui uma série de tipos de dados que podem ser designados às colunas dos DataFrames.
        </li>
    </ul>
</div>

<center> 
    <img src='data_types1.png'>
</center>

<h3 style='font-size:30px;font-style:italic'> Schemas and Creating DataFrames</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            O <em> schema</em> de um DataFrame é a relação do nome de suas colunas com os seus respectivos tipos de dados.
        </li>
        <li> 
            O Spark é capaz de inferir o schema de uma tabela, mas isso pode ser demorado e demandar uma alta capacidade de processamento do computador. Por isso, é recomendável que nós façamos essa tarefa.
        </li>
    </ul>
</div>

<p style='color:red'> Two ways to define a schema (fazer o código com ambas as abordagens)